In [ ]:
from qiskit.providers.fake_provider import FakeHanoi
from qiskit import QuantumCircuit, BasicAer
from qiskit.tools import job_monitor
from qiskit.quantum_info.analysis import hellinger_fidelity

from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q')


In [ ]:
from qiskit.providers.ibmq import least_busy

small_devices = provider.backends(filters=lambda x: x.configuration().n_qubits >= 7
                                   and not x.configuration().simulator)
backend = least_busy(small_devices)

print(backend)

In [ ]:
# BQSKIT Compile
from bqskit import Circuit
from bqskit.compiler import Compiler, compile
from qiskit.providers.fake_provider import FakeHanoi
from bqskit.ext import model_from_backend


machine_model = model_from_backend(backend)
machine_model.add_noise_model_from_backend(backend)


circ_name = "qft_5"
run_noise_aware = True
placement_name = "noise_aware" if run_noise_aware else "greedy"

# Load a circuit from QASM
circuit = Circuit.from_file('{}.qasm'.format(circ_name))

print(circuit.gate_counts)
print(circuit.depth)

# Compile it with optimization level 2
noise_aware_circuit = compile(circuit, model=machine_model, run_noise_aware=run_noise_aware, optimization_level=3, max_synthesis_size=4)
greedy_circuit = compile(circuit, model=machine_model, run_noise_aware=False, optimization_level=3, max_synthesis_size=4)

In [ ]:
from bqskit.ext import bqskit_to_qiskit

circ_name = "qft_5"

correct_circ = QuantumCircuit.from_qasm_file("{}.qasm".format(circ_name))
greedy_circ = bqskit_to_qiskit(greedy_circuit)
noise_circ = bqskit_to_qiskit(noise_aware_circuit)

greedy_circ.measure_all()
noise_circ.measure_all()

from qiskit import transpile
transpiled_circ = transpile(correct_circ, backend, routing_method='sabre')
transpiled_circ.measure_all()

print(greedy_circ.count_ops())
print(noise_circ.count_ops())
print(transpiled_circ.count_ops())

print(greedy_circ.depth())
print(noise_circ.depth())
print(transpiled_circ.depth())

all_circs = [transpiled_circ, greedy_circ, noise_circ]

In [ ]:
from qiskit import execute
job = execute(all_circs, backend, shots=1024)
job_monitor(job)

In [ ]:
counts_1 = job.result().get_counts(1)
print(counts_1)
counts_2 = job.result().get_counts(2)
print(counts_2)
counts_3 = job.result().get_counts(0)
print(counts_3)

In [ ]:
# Run Simulator
from qiskit.providers.aer import AerSimulator

simulator = AerSimulator(method='matrix_product_state')
job_4 = simulator.run(transpiled_circ)

transpiled_circ.count_ops()

In [ ]:
counts_actual = job_4.result().get_counts()
print(counts_actual)

In [ ]:
fid_noise_aware = hellinger_fidelity(counts_1, counts_actual)
fid_greedy = hellinger_fidelity(counts_2, counts_actual)
fid_norm = hellinger_fidelity(counts_3, counts_actual)

print("Noise Aware Fidelity: {}".format(fid_noise_aware))
print("Greedy Fidelity: {}".format(fid_greedy))
print("IBM Compiler Fidelity: {}".format(fid_norm))